### OBJ:
- get pydap access to OPENDAP site to get data
- get access to MULTIPLE files

In [1]:
import netCDF4 # packages to open 'netcdf' file
import numpy as np # numpy and pandas packages to pre-process the dataset
import pandas as pd
import matplotlib.pyplot as plt # to create plots and graphs
from mpl_toolkits.basemap import Basemap # to create geo-spatial map, requires dependencies installation
import plotly.express as px

# to load webcontent and retrieve data from link

from IPython.display import display, HTML
from IPython import display
from datetime import datetime

from IPython import display
import matplotlib.animation as animation

In [2]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib
import matplotlib as mpl

from matplotlib.collections import PatchCollection

## import function from other dir. for Visualization

In [3]:
# http://localhost:8888/notebooks/JPL_projects/jupyter_nb_codes/APX/Arset/plot_vertices_function.ipynb

In [4]:
# from plot_vertices_function import *

In [5]:
# !pip install dash_core_components

In [6]:
import dash
import dash_core_components as dcc
import dash_html_components as html

C:\Users\sagarl\AppData\Local\Temp\ipykernel_23144\954449312.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\sagarl\AppData\Local\Temp\ipykernel_23144\954449312.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [7]:
import re
# to grab data from entire year, month
from bs4 import BeautifulSoup # 

In [8]:
## Libraries
from urllib import request, parse
import getpass
import netrc
import os
import requests
import matplotlib.image as mpimg
import os
import time
from netCDF4 import Dataset

# pydap library to open session
from pydap import client
from pydap.cas.urs import setup_session
from pydap.client import open_url

import json
%matplotlib inline

### Func:
- get session key using the credentials

In [9]:
def get_session(url, file_name):
    """
    Creating a session with url and filename in openDap for data retrieval
    https://oco2.gesdisc.eosdis.nasa.gov/opendap/
    """
    try:
        login_credentials= 'uat.urs.earthdata.nasa.gov'
        username, _, password = netrc.netrc().authenticators(login_credentials)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('\n*******************************************\n')
        print('Please provide your Earthdata Login credentials to allow data access\n')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (url))
        print('\n')
        username = input('Username:')
        password = getpass.getpass()
        print('\n*******************************************\n')
        
    # pydap session
    session = setup_session(username, password, check_url= url + file_name)
    
    # using the session to get access the data
    return session

### DateTime:

In [10]:
def conv_date(d):
    return datetime.strptime(str(d), '%Y%m%d%H%M%S%f')    

### Open netCDF and conv. to CSV

In [11]:
def open_netcdf(url, file_name):
    
    # get session from the URL and filename
    session= get_session(url, file_name)
    
    # open the netcdf file and store in dataframe
    pydap_df= open_url(url+file_name, session = session)
    
    # Data preprocessing with new column names from the variables
    # Note: reformating requires for datetime
    
    df_xco2= pd.DataFrame(columns=[ "Latitude", "Longitude", "xco2", "xco2_quality_flag"])
    
    df_xco2["xco2"]= pydap_df["xco2"][:]
    #df_xco2["DateTime"]= pydap_df["date"][:]
    df_xco2["Latitude"]= pydap_df["latitude"][:]
    df_xco2["Longitude"]= pydap_df["longitude"][:]
    df_xco2["xco2_quality_flag"]= pydap_df["xco2_quality_flag"][:]
#     df_xco2["Longitude_vertices"]= pydap_df["vertex_longitude"][:]
#     df_xco2["Latitude_vertices"]= pydap_df["vertex_latitude"][:]
    
    # convert data type
    df_xco2= df_xco2.astype(float)
    
    return df_xco2.round(3)

### EXAMPLE:
- retrieve SINGLE file from OpenDap 

## B11 ver:
- https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Standard.11r/contents.html

In [12]:
url='https://oco2.gesdisc.eosdis.nasa.gov/'
file_name= 'opendap/OCO2_L2_Lite_FP.10r/2020/oco2_LtCO2_200109_B10206Ar_200728203551s.nc4'

In [13]:
url+file_name

'https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.10r/2020/oco2_LtCO2_200109_B10206Ar_200728203551s.nc4'

In [15]:
# open the netcdf file and store in dataframe
pydap_df= open_netcdf(url, file_name)

print('*******************************\n')
print("Printing the First three rows:\n")
pydap_df.head(3)


*******************************************

Please provide your Earthdata Login credentials to allow data access

Your credentials will only be passed to https://oco2.gesdisc.eosdis.nasa.gov/ and will not be exposed in Jupyter


Username:sagarlimbu
········

*******************************************

*******************************

Printing the First three rows:



,Latitude,Longitude,xco2,xco2_quality_flag
0,-82.680,-120.762,410.116,1.0
1,-82.668,-120.859,409.631,1.0
2,-76.281,-144.006,409.603,1.0


### BASEMAP patcehs

In [16]:
def get_patches(vertices_pack ):#, season):

    # for CSV files from path
#     all_files = glob.glob(os.path.join(path_, "*.csv"))
#     df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
    
    months= ['Jan', 'Feb', 'March','April','May', 'June', 'July', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']
    months_num= np.arange(1, 13).tolist()
    
    ## XCO2 total values
#     xco2_total= np.array(df['Xco2'][:])

#     ## FUNCTION to split the brackets
#     def split_bracket(d):
#         l = [float(x.strip(' []')) for x in d.split(',')]
#         return l

#     ## pre-processing the vertices columnx
#     df["Longitude_vertices"]=df["Longitude_vertices"].apply(split_bracket)
#     df["Latitude_vertices"]=df["Latitude_vertices"].apply(split_bracket)

    spring= np.arange(3, 6)
    fall= np.arange(9, 12)
   
    
#     if str(season) == "spring":
#         df= df[df["Month"].isin(spring)]
#         months_selected= spring
#     elif str(season) == "fall":
#         df= df[df["Month"].isin(fall)]
#         months_selected= fall
        
        
    # reset the index
#     df= df.reset_index()
#     df= df.drop(columns=['index'])
    
#     # DSTACK the vertices together
#     flat_vert=[]
#     for j in range(len(df)):
#         flat_vert.append(np.dstack([ np.array(df['Longitude_vertices'][j]), np.array(df['Latitude_vertices'][j]) ]))

#     # flat_verterices SHAPE from MULTI-dimensional shape
#     unpack_vert= [element for sublist in flat_vert for element in sublist]

    ## Combined the polygon patches total
    # POLYGONS for matplotlib
    patches_total= []

    for row in vertices_pack:
        polygon= mpatches.Polygon(row)
        patches_total.append(polygon)

#     df_months= df["Month"].unique()
        
    return patches_total #, [months[m] for m in months_selected]

## Verticies

In [17]:
def get_products(url, file_name):
    
    # create session
    # get session from the URL and filename
    session= get_session(url, file_name)
    
    # open the netcdf file and store in dataframe
    df= open_url(url+file_name, session = session)
    
    countFiles=0

    lon_list= []
    lat_list= []
    sif_list= []
    qual_flag_list= []
    lat=[]
    lon=[]

#     for j in file_names:
#         if j.endswith(".nc4"):
#     var_= nc.Dataset(j)

    lon_list.append(np.array(df['vertex_longitude'][:]).tolist())
    lat_list.append(np.array(df['vertex_latitude'][:]).tolist())
    sif_list.append(np.array(df['xco2'][:]).tolist())
    qual_flag_list.append(np.array(df['xco2_quality_flag'][:]).tolist())

    # lat and long without vert
    lat.append(np.array(df['latitude'][:]).tolist())
    lon.append(np.array(df['longitude'][:]).tolist())
            
    return lon_list, lat_list, sif_list, qual_flag_list, lat, lon

In [195]:
lon_list, lat_list, xco2_list, qual_flag_list, lat, lon= get_products(url, file_name)


*******************************************

Please provide your Earthdata Login credentials to allow data access

Your credentials will only be passed to https://oco2.gesdisc.eosdis.nasa.gov/ and will not be exposed in Jupyter


Username:sagarlimbu
········

*******************************************



In [197]:
lon_list= [elem for sublist in lon_list for elem in sublist]
lat_list= [elem for sublist in lat_list for elem in sublist]
xco2_list= [elem for sublist in xco2_list for elem in sublist]
qual_flag_list= [elem for sublist in lon_list for elem in sublist]
lat= [elem for sublist in lat for elem in sublist]
lon= [elem for sublist in lon for elem in sublist]

In [198]:
lon_list[0]

[-120.62397003173828,
 -120.72196197509766,
 -120.86984252929688,
 -120.77220153808594]

In [218]:
total_row= len(lon_list)
flat_vert=[]
for j in range((total_row)):
#     flat_vert.append(np.dstack([ np.array(df['Longitude_vertices'][j]), np.array(df['Latitude_vertices'][j]) ]))
     flat_vert.append(np.dstack([ np.array(lon_list[j]), np.array(lat_list[j]) ]))


In [205]:
flat_vert= [elem for sublist in flat_vert for elem in sublist]

In [220]:
# for r in flat_vert:
#     print(r)

In [221]:
# %%time
# patches_total = get_patches(flat_vert)

In [ ]:

## plot the xco2
plot_monthly(patches_total, xco2_total, 2020, '###', months_sel)

## Creating REGEX to scrap required file
- by year, version

In [19]:
"https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.11r/2019/oco2_LtCO2_190107_B11014Ar_221017171018s.nc4.html"

print("Enter the Year:")
year= input()
print("\nEnter the Month:")
month= input()
print("\nEnter the version:")
ver_= input()

lite_file= '/OCO2_L2_Lite_FP.'+ str(ver_)+ 'r' +'/'+ year 
#+ '/' + 'oco2_LtCO2' 

print('****************************\n')
print(lite_file)

Enter the Year:
2021

Enter the Month:
08

Enter the version:
11
****************************

/OCO2_L2_Lite_FP.11r/2021


# web scrap

In [87]:
"https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.11r/2019/oco2_LtCO2_190107_B11014Ar_221017171018s.nc4.html"

print("Enter the Year:")
year= input()
print("\n*****************************************\n")
print("\nEnter the Month:")
month= input()
print("\n*****************************************\n")
print("\nEnter the version:")
ver_= input()
print("\n*****************************************\n")

lite_file= '/OCO2_L2_Lite_FP.'+ str(ver_)+ 'r' +'/'+ year 
#+ '/' + 'oco2_LtCO2' 

main_url='https://oco2.gesdisc.eosdis.nasa.gov/opendap'
content= '/contents.html'

s= requests.get(main_url + lite_file + content)

# Scrap the content by specified URL
soup= BeautifulSoup(s.content, 'html.parser')

# Regex applying on href LINK
# Beautiful soup accepts href 
list_files=[]
# html_links= soup.select('a[href$=".nc4.html"]')


html_links= soup.select('a[href$=".nc4.html"]')

for link in html_links:
    list_files.append(link['href'])
    print(link['href'])
    
    
## CLEAN the files
# removing last strings '.html' to download the files from PYDAP library
files_oco2= [f[:-5] for f in list_files]

Enter the Year:
2019

*****************************************


Enter the Month:
06

*****************************************


Enter the version:
10

*****************************************

oco2_LtCO2_190101_B10206Ar_200729172616s.nc4.html
oco2_LtCO2_190101_B10206Ar_200729172616s.nc4.html
oco2_LtCO2_190102_B10206Ar_200729172942s.nc4.html
oco2_LtCO2_190102_B10206Ar_200729172942s.nc4.html
oco2_LtCO2_190103_B10206Ar_200729173012s.nc4.html
oco2_LtCO2_190103_B10206Ar_200729173012s.nc4.html
oco2_LtCO2_190104_B10206Ar_200729173043s.nc4.html
oco2_LtCO2_190104_B10206Ar_200729173043s.nc4.html
oco2_LtCO2_190105_B10206Ar_200729173113s.nc4.html
oco2_LtCO2_190105_B10206Ar_200729173113s.nc4.html
oco2_LtCO2_190106_B10206Ar_200729173145s.nc4.html
oco2_LtCO2_190106_B10206Ar_200729173145s.nc4.html
oco2_LtCO2_190107_B10206Ar_200729173429s.nc4.html
oco2_LtCO2_190107_B10206Ar_200729173429s.nc4.html
oco2_LtCO2_190108_B10206Ar_200729173516s.nc4.html
oco2_LtCO2_190108_B10206Ar_200729173516s.nc4.html
oco

In [85]:
list_files[:4]

['oco2_LtCO2_200101_B10206Ar_200728183348s.nc4.html',
 'oco2_LtCO2_200101_B10206Ar_200728183348s.nc4.html',
 'oco2_LtCO2_200102_B10206Ar_200728203252s.nc4.html',
 'oco2_LtCO2_200102_B10206Ar_200728203252s.nc4.html']

In [83]:
files_oco2[:6]

['oco2_LtCO2_200101_B10206Ar_200728183348s.nc4',
 'oco2_LtCO2_200101_B10206Ar_200728183348s.nc4',
 'oco2_LtCO2_200102_B10206Ar_200728203252s.nc4',
 'oco2_LtCO2_200102_B10206Ar_200728203252s.nc4',
 'oco2_LtCO2_200103_B10206Ar_200728203534s.nc4',
 'oco2_LtCO2_200103_B10206Ar_200728203534s.nc4']

In [68]:
platform= 'oco2_LtCO2_'+ str(year[:-2]) + str(month)
platform

'oco2_LtCO2_2005'

In [79]:
tot_months= [ x for x in files_oco2 if x.startswith(platform)]
len(tot_months)

62

In [80]:
tot_months

['oco2_LtCO2_200501_B11014Ar_220825221036s.nc4',
 'oco2_LtCO2_200501_B11014Ar_220825221036s.nc4',
 'oco2_LtCO2_200502_B11014Ar_220825221106s.nc4',
 'oco2_LtCO2_200502_B11014Ar_220825221106s.nc4',
 'oco2_LtCO2_200503_B11014Ar_220825221136s.nc4',
 'oco2_LtCO2_200503_B11014Ar_220825221136s.nc4',
 'oco2_LtCO2_200504_B11014Ar_220825221212s.nc4',
 'oco2_LtCO2_200504_B11014Ar_220825221212s.nc4',
 'oco2_LtCO2_200505_B11014Ar_220825221242s.nc4',
 'oco2_LtCO2_200505_B11014Ar_220825221242s.nc4',
 'oco2_LtCO2_200506_B11014Ar_220825221312s.nc4',
 'oco2_LtCO2_200506_B11014Ar_220825221312s.nc4',
 'oco2_LtCO2_200507_B11014Ar_220825221342s.nc4',
 'oco2_LtCO2_200507_B11014Ar_220825221342s.nc4',
 'oco2_LtCO2_200508_B11014Ar_220825221413s.nc4',
 'oco2_LtCO2_200508_B11014Ar_220825221413s.nc4',
 'oco2_LtCO2_200509_B11014Ar_220825221443s.nc4',
 'oco2_LtCO2_200509_B11014Ar_220825221443s.nc4',
 'oco2_LtCO2_200510_B11014Ar_220825221518s.nc4',
 'oco2_LtCO2_200510_B11014Ar_220825221518s.nc4',
 'oco2_LtCO2_200511_